In [358]:
from torchvision import models

In [359]:
model = models.resnet18(pretrained=True)

In [360]:
n_channels = 6

In [361]:
a = model.conv1.weight.data

In [362]:
b = model.conv1.weight.data[:, :int(n_channels - 3), :, :]

In [363]:
c = torch.cat([a, b], dim=1)

In [364]:
c.shape

torch.Size([64, 6, 7, 7])

In [355]:
b.shape

torch.Size([64, 6, 7, 7])